In [ ]:
######################## BUILDING CNN AWL MODEL 5ALES  ###############################

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms 


class Net(nn.Module):
    def __init__(self):
        
        super(Net,self).__init__() # calling super constructor
        
        
        #self.norm = self.norm
        self.conv1 = nn.Conv1d(6,196,16,stride=1) #input and output input 1 it outputs 196 feature maps and 16 is kernel size
        
        x = torch.rand(768)  #creating random data to pass to conv layer to know the dimension it will pass so we can pass it to feed forward layer
        self.conv2_drop = nn.Dropout2d(p=0.05) #dropout
        self._to_linear= None #null
        
        #self.convs(self.norm(x))  #pass input to conv layers then get dimension and set variable of input in forward layers
        
        self.fc1 = nn.Linear(5488,6)
        #print(self.fc1)
        #self.fc2 = nn.Linear(512,2)  #cuz we have only 6 classes
        
  
  
   
      
        
    def convs(self,x):
        #x = self.norm(x)
        #print(x)
        x= F.max_pool1d(F.relu(self.conv1(x)),(4))   #applying max pooling to output of conv layers size of pooling layer
       
        #if self._to_linear is None:
            #self._to_linear = x[0].shape[0]*x[0].shape[1]
            #print(self._to_linear)
        #print(x[0].shape)
        
        return x
     
        
     
    
    def forward(self,x):
        x = x.reshape(-1,6,128)
        x = self.convs(x) #pass to conv
        x = x.view(-1,5488) #flatten it
        #print(self._to_linear)
        x = F.relu(self.fc1(x))# add dropout before softmax
        x = self.conv2_drop(x)
        return F.softmax(x,dim=1)
        
    

net = Net()
#x = torch.rand(768)
#print(torch.argmax(net(x)))
#print(net)





        
############### TRAIN #####################
optimizer = optim.Adam(net.parameters(),lr=5e-4,weight_decay=5e-4) #reguralization
loss_function = nn.MSELoss()


########## LOADING SEGMENTS AND LABELS ##############
training_data = np.load("training_data.npy",allow_pickle=True)
#print(training_data)



#training_labels = np.load("training_labels.npy",allow_pickle=True)


X = torch.Tensor([i[0] for i in training_data]) #getting input and converting into a tensor flatten input


Y = torch.Tensor([i[1] for i in training_data ])
#print(len(X[78]))
#print(len(X[0]))
BATCH_SIZE = 200
EPOCHS = 200

for epochs in range(EPOCHS):
    for i in tqdm(range(0,len(X),BATCH_SIZE)):
        batch_X = X[i:i+BATCH_SIZE]  #i,i+batchsize = 0 100 100 200 200 300
        #print(type(batch_X).__name__)
        batch_y = Y[i:i+BATCH_SIZE]
        net.zero_grad()
        #print(batch_X.shape)
        #print(len(batch_X))
        #print(batch_X)
        #print(len(batch_X[125]))
        outputs = net(batch_X)
        
        loss = loss_function(outputs,batch_y)
        loss.backward()
        optimizer.step()
        #break



        
    

######## SAVING OUR TRAINED MODEL
FILE = "model6inputsreshape200ep.pth"

#print(net)
torch.save(net.state_dict(),FILE)    


















###### LOADING SAVED PRE TRAINED MODEL ##########
#import numpy as np
net = Net()
net.load_state_dict(torch.load(FILE))
#x = torch.rand(128)
#print(x)
#net(x)
#print(torch.argmax(net(x)))
#print(x[0][0])






####### TESTING LOADED MODEL ############

testing_data = np.load("testing_data.npy",allow_pickle=True)
#testing_labels = np.load("testing_labels.npy",allow_pickle=True)
test_X = torch.Tensor([i[0] for i in testing_data]) #getting input and converting into a tensor flatten input
#print(test_X[1][5])
#labels = np.load("testing_labels.npy",allow_pickle=True)

test_y = torch.Tensor([i[1] for i in testing_data ])
total = 0
correct =0
totalWalking =0
correctWalking = 0
totalWalkingUp =0
correctWalkingUp = 0
totalWalkingDown =0
correctWalkingDown = 0
totalSitting =0
correctSitting = 0
totalStanding =0
correctStanding = 0
totalLaying =0
correctLaying = 0
real=7
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        if real_class==0:
            totalWalking+=1
            real=0
        elif real_class==1:
            totalWalkingUp+=1
            real=1
        elif real_class==2:
            totalWalkingDown+=1
            real=2
        elif real_class==3:
            totalSitting+=1
            real=3
        elif real_class==4:
            totalStanding+=1
            real=4
      
        elif real_class==5:
            totalLaying+=1
            real=5
            
        #print(real_class)
        #print(test_X[i].view(-1,6,128))
        net_out = net(test_X[i])[0]
        #print(net_out)
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct+=1
            if real==0:
                correctWalking+=1
            if real==1:
                correctWalkingUp+=1
            if real==2:
                correctWalkingDown+=1
            if real==3:
                correctSitting+=1
            if real==4:
                correctStanding+=1
            if real==5:
                correctLaying+=1
        total+=1

print(round(correct/total,3)*100)
print(round(correctWalking/totalWalking,3)*100)
print(round(correctWalkingUp/totalWalkingUp,3)*100)
print(round(correctWalkingDown/totalWalkingDown,3)*100)
print(round(correctSitting/totalSitting,3)*100)
print(round(correctStanding/totalStanding,3)*100)
print(round(correctLaying/totalLaying,3)*100)



#### DEPLOYING MODEL TO ANDROID EMULATOR #########

net = Net()
net.load_state_dict(torch.load(FILE))
net.eval()
x = []
for i in range(128):
    x.append(1)
print(x)
y=[]
y.append(x)
y.append(x)
y.append(x)
y.append(x)
y.append(x)
y.append(x)
z = torch.Tensor(y)
    
#print(x)
print(torch.argmax(net(z)))
traced_script_module = torch.jit.trace(net, z)
traced_script_module.save("/Users/macbookair/Desktop/android-demo-app/HelloWorldApp/app/src/main/assets/final.pt") # saving to assets folder in android studio
#USED FOR EMULATOR



